# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [ ]:
# Instalace spaCy z Jupyter Notebooku
import sys
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

In [22]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [36]:
import csv
import os.path
def filetoLower(inf:str, outf:str, remove = False):
    if (not os.path.exists(outf)) or remove:
        with open(inf,"r",encoding="utf-8") as file, open(outf,"w",encoding="utf-8") as outFile:
            reader = csv.reader(file)
            writer = csv.writer(outFile)
            writer.writerow(next(reader))
            for line in reader:
                outFile.write(line.lower())
    else:
        print("file already exsists")
filetoLower("articles.csv","articlesLC.csv",True)

In [37]:
disabledChars = [',','.','\'','"','’','‘','(',')','“','”','?','/','\\','!','[',']','–','/','\\',':','-']
def filetoRemInter(inf:str, outf:str, remove = False):
    if (not os.path.exists(outf)) or remove:
        with open(inf,"r",encoding="utf-8") as file, open(outf,"w",encoding="utf-8") as outFile:
            for line in file:
                for ch in disabledChars:
                    line = line.replace(ch,' ')
                line = line.replace('  ',' ')
                outFile.write(line)
    else:
        print("file already exsists")
filetoRemInter("articlesLC.csv","articlesLC_RI.csv",True)

In [32]:
def fileLematize(inf:str, outf:str, remove = False):
    if (not os.path.exists(outf)) or remove:
        with open(inf,"r",encoding="utf-8") as file, open(outf,"w",encoding="utf-8") as outFile:
            for line in file:
                line = " ".join([token.lemma_ for token in lemmatizer(line)])
                outFile.write(line)
    else:
        print("file already exsists")
fileLematize("articlesLC_RI.csv","articlesLEMMA.csv",True)

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)